In [67]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import tensorflow as tf 
import re 
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns 
from sklearn.preprocessing import MinMaxScaler
plt.style.use('ggplot')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [68]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense, Embedding, LSTM, Bidirectional, Dropout, Concatenate
from keras.preprocessing.text import Tokenizer
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [4]:
df = pd.read_csv("C:/Users/natas/OneDrive/Documents/UCD/Sem 3/TruthSeeker2023/Features_For_Traditional_ML_Techniques.csv")

In [5]:
df.columns

Index(['Unnamed: 0', 'majority_target', 'statement', 'BinaryNumTarget',
       'tweet', 'followers_count', 'friends_count', 'favourites_count',
       'statuses_count', 'listed_count', 'following', 'embeddings', 'BotScore',
       'BotScoreBinary', 'cred', 'normalize_influence', 'mentions', 'quotes',
       'replies', 'retweets', 'favourites', 'hashtags', 'URLs', 'unique_count',
       'total_count', 'ORG_percentage', 'NORP_percentage', 'GPE_percentage',
       'PERSON_percentage', 'MONEY_percentage', 'DATE_percentage',
       'CARDINAL_percentage', 'PERCENT_percentage', 'ORDINAL_percentage',
       'FAC_percentage', 'LAW_percentage', 'PRODUCT_percentage',
       'EVENT_percentage', 'TIME_percentage', 'LOC_percentage',
       'WORK_OF_ART_percentage', 'QUANTITY_percentage', 'LANGUAGE_percentage',
       'Word count', 'Max word length', 'Min word length',
       'Average word length', 'present_verbs', 'past_verbs', 'adjectives',
       'adverbs', 'adpositions', 'pronouns', 'TOs', 'deter

In [6]:
df.dtypes

Unnamed: 0           int64
majority_target       bool
statement           object
BinaryNumTarget    float64
tweet               object
                    ...   
ampersand            int64
capitals             int64
digits               int64
long_word_freq       int64
short_word_freq      int64
Length: 64, dtype: object

In [7]:
scaler = MinMaxScaler()

In [8]:
df[['BotScore', 'quotes' , 'cred', 'mentions', 'replies', 'retweets', 'hashtags', 'URLs', 'unique_count' , 'normalize_influence', 'past_verbs', 'adjectives' , 'NORP_percentage', 'PERSON_percentage', 'CARDINAL_percentage', 'ORG_percentage', 'MONEY_percentage', 'PRODUCT_percentage', 'EVENT_percentage' , 'WORK_OF_ART_percentage' , 'Word count', 'Average word length']] = scaler.fit_transform(df[['BotScore', 'quotes', 'cred', 'mentions', 'replies', 'retweets', 'hashtags', 'URLs', 'unique_count' , 'normalize_influence', 'past_verbs', 'adjectives' , 'NORP_percentage', 'PERSON_percentage', 'CARDINAL_percentage', 'ORG_percentage', 'MONEY_percentage', 'PRODUCT_percentage', 'EVENT_percentage' , 'WORK_OF_ART_percentage' , 'Word count', 'Average word length']])

In [9]:
df.head()

,Unnamed: 0,majority_target,statement,BinaryNumTarget,tweet,followers_count,friends_count,favourites_count,statuses_count,listed_count,...,determiners,conjunctions,dots,exclamation,questions,ampersand,capitals,digits,long_word_freq,short_word_freq
0,0,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,4262.0,3619.0,34945.0,16423.0,44.0,...,0,0,5,0,1,0,33,3,5,19
1,1,True,End of eviction moratorium means millions of A...,1.0,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,1393.0,1621.0,31436.0,37184.0,64.0,...,0,2,1,0,0,0,14,0,2,34
2,2,True,End of eviction moratorium means millions of A...,1.0,THE SUPREME COURT is siding with super rich pr...,9.0,84.0,219.0,1184.0,0.0,...,0,1,0,0,0,0,3,0,4,10
3,3,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders\n\nBroken campaign promi...,4262.0,3619.0,34945.0,16423.0,44.0,...,0,1,3,0,0,1,6,8,1,30
4,4,True,End of eviction moratorium means millions of A...,1.0,@OhComfy I agree. The confluence of events rig...,70.0,166.0,15282.0,2194.0,0.0,...,0,1,3,0,1,0,11,3,2,19


In [10]:
#df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,134198.0,67098.500000,38739.770050,0.0,33549.250000,67098.500000,100647.750000,134197.0
BinaryNumTarget,134198.0,0.513644,0.499816,0.0,0.000000,1.000000,1.000000,1.0
followers_count,134198.0,11293.082103,437497.073676,0.0,70.000000,354.000000,1573.000000,130601913.0
friends_count,134198.0,1893.454455,6997.695671,0.0,168.000000,567.000000,1726.000000,586901.0
favourites_count,134198.0,32981.233878,68780.210325,0.0,1356.000000,8377.000000,33526.500000,1765080.0
statuses_count,134198.0,34195.761449,75101.195265,1.0,3046.000000,11019.000000,33573.750000,2958918.0
listed_count,134198.0,73.300198,1083.274277,0.0,0.000000,2.000000,11.000000,222193.0
following,134198.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
BotScore,134198.0,0.059106,0.167819,0.0,0.030000,0.030000,0.030000,1.0
BotScoreBinary,134198.0,0.032355,0.176942,0.0,0.000000,0.000000,0.000000,1.0


In [11]:
df['text'] = df['statement'] + df['tweet']
df.drop('statement', axis=1, inplace=True)
df.drop('tweet', axis=1, inplace=True)

In [12]:
df.columns

Index(['Unnamed: 0', 'majority_target', 'BinaryNumTarget', 'followers_count',
       'friends_count', 'favourites_count', 'statuses_count', 'listed_count',
       'following', 'embeddings', 'BotScore', 'BotScoreBinary', 'cred',
       'normalize_influence', 'mentions', 'quotes', 'replies', 'retweets',
       'favourites', 'hashtags', 'URLs', 'unique_count', 'total_count',
       'ORG_percentage', 'NORP_percentage', 'GPE_percentage',
       'PERSON_percentage', 'MONEY_percentage', 'DATE_percentage',
       'CARDINAL_percentage', 'PERCENT_percentage', 'ORDINAL_percentage',
       'FAC_percentage', 'LAW_percentage', 'PRODUCT_percentage',
       'EVENT_percentage', 'TIME_percentage', 'LOC_percentage',
       'WORK_OF_ART_percentage', 'QUANTITY_percentage', 'LANGUAGE_percentage',
       'Word count', 'Max word length', 'Min word length',
       'Average word length', 'present_verbs', 'past_verbs', 'adjectives',
       'adverbs', 'adpositions', 'pronouns', 'TOs', 'determiners',
       'conju

In [13]:
df.pop('Unnamed: 0')
df.pop('majority_target')
df.pop('embeddings')

0         [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1         [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
2         [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
3         [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
4         [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
                          ...                  
134193    [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
134194    [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
134195    [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
134196    [[0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1]]
134197    [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Name: embeddings, Length: 134198, dtype: object

In [14]:
y = df['BinaryNumTarget']

In [15]:
df.pop('BinaryNumTarget')

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
134193    0.0
134194    0.0
134195    0.0
134196    0.0
134197    0.0
Name: BinaryNumTarget, Length: 134198, dtype: float64

In [17]:
X = df[['text', 'quotes', 'BotScore', 'cred', 'mentions', 'replies', 'retweets', 'hashtags', 'URLs', 'unique_count' , 'normalize_influence', 'past_verbs', 'adjectives' , 'NORP_percentage', 'PERSON_percentage', 'CARDINAL_percentage', 'ORG_percentage', 'MONEY_percentage', 'PRODUCT_percentage', 'EVENT_percentage' , 'WORK_OF_ART_percentage' , 'Word count', 'Average word length']].copy()

In [51]:
X

,text,quotes,BotScore,cred,mentions,replies,retweets,hashtags,URLs,unique_count,...,NORP_percentage,PERSON_percentage,CARDINAL_percentage,ORG_percentage,MONEY_percentage,PRODUCT_percentage,EVENT_percentage,WORK_OF_ART_percentage,Word count,Average word length
0,end eviction moratorium means millions america...,0.000169,0.03,0.540794,0.083333,0.000024,0.000024,0.0,1.0,0.555556,...,0.166667,0.166667,0.0,0.333333,0.166667,0.0,0.0,0.0,0.381818,0.072766
1,end eviction moratorium means millions america...,0.000000,0.03,0.462177,0.250000,0.000000,0.000000,0.0,1.0,0.222222,...,0.250000,0.750000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.463636,0.063230
2,end eviction moratorium means millions america...,0.000338,0.03,0.096774,0.000000,0.000119,0.000000,0.0,1.0,0.111111,...,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.236364,0.079084
3,end eviction moratorium means millions america...,0.000000,0.03,0.540794,0.083333,0.000000,0.000000,0.0,1.0,0.444444,...,0.333333,0.166667,0.0,0.000000,0.166667,0.0,0.0,0.0,0.445455,0.063518
4,end eviction moratorium means millions america...,0.000000,0.03,0.296610,0.083333,0.000000,0.000000,0.0,1.0,0.333333,...,0.000000,0.000000,0.0,0.333333,0.333333,0.0,0.0,0.0,0.300000,0.068778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134193,joe bidens great grandfather joseph j biden sl...,0.000169,0.03,0.438041,0.000000,0.000024,0.000008,0.0,1.0,0.111111,...,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.0,0.454545,0.066292
134194,joe bidens great grandfather joseph j biden sl...,0.000169,0.03,0.501533,0.000000,0.000000,0.000008,0.0,0.0,0.222222,...,0.000000,0.500000,0.0,0.500000,0.000000,0.0,0.0,0.0,0.172727,0.078060
134195,joe bidens great grandfather joseph j biden sl...,0.000000,0.03,0.102619,0.083333,0.000000,0.000000,0.0,1.0,0.222222,...,0.000000,0.000000,0.0,0.000000,0.500000,0.0,0.0,0.0,0.490909,0.058677
134196,joe bidens great grandfather joseph j biden sl...,0.000000,0.03,0.042553,0.083333,0.000000,0.000000,0.0,1.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.136364,0.071161


In [21]:
nltk.download('stopwords')

# Clean the text column
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Join the tokens back into a single string
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\natas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Apply the clean_text function to the 'text' column
X['text'] = X['text'].apply(clean_text)

In [50]:
X['text']

0         end eviction moratorium means millions america...
1         end eviction moratorium means millions america...
2         end eviction moratorium means millions america...
3         end eviction moratorium means millions america...
4         end eviction moratorium means millions america...
                                ...                        
134193    joe bidens great grandfather joseph j biden sl...
134194    joe bidens great grandfather joseph j biden sl...
134195    joe bidens great grandfather joseph j biden sl...
134196    joe bidens great grandfather joseph j biden sl...
134197    joe bidens great grandfather joseph j biden sl...
Name: text, Length: 134198, dtype: object

In [32]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['text'])
#X_train_sequences = tokenizer.texts_to_sequences(X_train['text'])
#X_test_sequences = tokenizer.texts_to_sequences(X_test['text'])

In [33]:
#tokenizer = Tokenizer()
#tokenizer.fit_on_texts(X['text'])
X_sequences = tokenizer.texts_to_sequences(X['text'])
X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_sequences, maxlen=max_sequence_length)

In [46]:
X_padded

array([[    0,     0,     0, ...,   848,   795,  3053],
       [    0,     0,     0, ...,  1170, 17760,  2003],
       [    0,     0,     0, ...,   856,  1486,     4],
       ...,
       [    0,     0,     0, ...,  4819,    11,    74],
       [    0,     0,     0, ...,    26,    33,   150],
       [    0,     0,     0, ...,   296,    16,    63]])

In [52]:
X_additional = X[['quotes', 'BotScore', 'cred', 'mentions', 'replies', 'retweets', 'hashtags', 'URLs', 'unique_count',
                  'normalize_influence', 'past_verbs', 'adjectives', 'NORP_percentage', 'PERSON_percentage',
                  'CARDINAL_percentage', 'ORG_percentage', 'MONEY_percentage', 'PRODUCT_percentage',
                  'EVENT_percentage', 'WORK_OF_ART_percentage', 'Word count', 'Average word length']]

In [53]:
# Convert additional columns to numpy array
X_additional = np.array(X_additional)

In [54]:
# Combine text data and additional columns
X_combined = np.concatenate((X_padded, X_additional), axis=1)

In [55]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

In [34]:
#X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

In [26]:
# Pad the sequences to have the same length
#max_sequence_length = max(len(sequence) for sequence in X_train_sequences)
#X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_sequences, maxlen=max_sequence_length)
#X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_sequences, maxlen=max_sequence_length)

In [27]:
# Define the model
#model = Sequential()
#model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_sequence_length))
#model.add(Bidirectional(LSTM(64)))
#model.add(Dense(1, activation='sigmoid'))

In [56]:
# Define the model
#model = Sequential()
#model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_sequence_length))
#model.add(Bidirectional(LSTM(64, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))))
#model.add(Dropout(0.5))
#model.add(Dense(1, activation='sigmoid'))

In [57]:
#model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 111, 100)          13281600  
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              84480     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 13,366,209
Trainable params: 13,366,209
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [59]:
# Compile the model
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [77]:
# Train the model
#model.fit(X_train_padded, y_train, validation_data=(X_test_padded, y_test), epochs=10, batch_size=64)

In [60]:
# Define early stopping callback
#early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [76]:
# Train the model with early stopping
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64, callbacks=[early_stopping])

In [78]:
# Define the model
#The model architecture is defined using the functional API of Keras, allowing for multiple input layers.
#Two input layers are defined: input_text for the tokenized and padded text data, and 
#input_additional for the additional columns.
    
input_text = Input(shape=(max_sequence_length,))
input_additional = Input(shape=(X_additional.shape[1],))

In [79]:
embedding = Embedding(len(tokenizer.word_index) + 1, 100)(input_text)
lstm = Bidirectional(LSTM(64, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))(embedding)

concatenated = Concatenate()([lstm, input_additional])
dropout = Dropout(0.5)(concatenated)
output = Dense(1, activation='sigmoid')(dropout)

model = Model(inputs=[input_text, input_additional], outputs=output)

In [80]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [66]:
# Train the model with early stopping
model.fit([X_train[:, :max_sequence_length], X_train[:, max_sequence_length:]], y_train,
          validation_data=([X_test[:, :max_sequence_length], X_test[:, max_sequence_length:]], y_test),
          epochs=10, batch_size=64, callbacks=[early_stopping])

Epoch 1/10
1678/1678 [==============================] - 275s 162ms/step - loss: 0.2024 - accuracy: 0.9737 - val_loss: 0.0470 - val_accuracy: 0.9978
Epoch 2/10
1678/1678 [==============================] - 266s 159ms/step - loss: 0.0400 - accuracy: 0.9981 - val_loss: 0.0239 - val_accuracy: 0.9990
Epoch 3/10
1678/1678 [==============================] - 265s 158ms/step - loss: 0.0374 - accuracy: 0.9976 - val_loss: 0.0211 - val_accuracy: 0.9987
Epoch 4/10
1678/1678 [==============================] - 297s 177ms/step - loss: 0.0215 - accuracy: 0.9989 - val_loss: 0.0325 - val_accuracy: 0.9981
Epoch 5/10
1678/1678 [==============================] - 309s 184ms/step - loss: 0.0169 - accuracy: 0.9994 - val_loss: 0.0117 - val_accuracy: 0.9990
Epoch 6/10
1678/1678 [==============================] - 297s 177ms/step - loss: 0.0097 - accuracy: 0.9998 - val_loss: 0.0169 - val_accuracy: 0.9979
Epoch 7/10
1678/1678 [==============================] - 297s 177ms/step - loss: 0.0069 - accuracy: 0.9999 - val_

In [69]:
y_pred = model.predict([X_test[:, :max_sequence_length], X_test[:, max_sequence_length:]])

839/839 [==============================] - 17s 19ms/step


In [70]:
y_pred_labels = (y_pred > 0.5).astype(int)

In [71]:
accuracy = accuracy_score(y_test, y_pred_labels)

In [72]:
precision = precision_score(y_test, y_pred_labels)

In [73]:
recall = recall_score(y_test, y_pred_labels)

In [74]:
f1 = f1_score(y_test, y_pred_labels)

In [75]:
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

Accuracy: 0.9989567809239941
Precision: 0.9990553698590321
Recall: 0.9989102005231038
F1-Score: 0.9989827799171692
